# Corner detection

In this notebook, I demonstrate how to use OpenCV's features to detect corners.

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <vector>
#include <tuple>
#include <stdexcept>
// #include <sstream>
// #include <string>
#include <limits>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Open the webcam

We are going to use the [`cv::VideoCapture` class](https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html) to grab frames from the webcam.

In [5]:
// Open the default camera (see the 0 below)
VideoCapture webcam(0);

### Any error? Is the data loaded?

In [6]:
if (!webcam.isOpened())
    throw runtime_error("OpenCV found no webcam, the program will terminate");

## 3. Run the event loop

Create a function to perform the detection, description and matching

In [7]:
void detectCorners(int aSliderValue,
                  const Mat& aNewTestImage,
                  Mat& anImageWithCorners,
                  Mat& aCornerDetection,
                  int blockSize = 8,
                  int apertureSize = 11,
                  double k = 0.06)
{
    // Colour to greyscale
    Mat grey_image;
    cvtColor(aNewTestImage, grey_image, COLOR_BGR2GRAY);
    
    // Blur the image
    Mat image_blurred;
    medianBlur(grey_image, image_blurred, 5);
    
    // Detecting corners
    Mat dst;
    cornerHarris( image_blurred, dst, blockSize, apertureSize, k, BORDER_DEFAULT );
   
    // Normalizing
    normalize( dst, aCornerDetection, 0, 255, NORM_MINMAX, CV_8UC1, Mat() );
   
    // Draw a circle around corners
    anImageWithCorners = aNewTestImage;
    for( int j = 0; j < aCornerDetection.rows ; j++ )
    {
        for( int i = 0; i < aCornerDetection.cols; i++ )
        {
            if( (int) aCornerDetection.at<unsigned char>(j,i) > aSliderValue )
            {
                circle( anImageWithCorners, Point( i, j ), 1, Scalar( 0, 0, 255 ), FILLED, LINE_8 );
            }
        }
    }
}

Create a callback for the trackbar to control the threshold

In [8]:
int slider_value = 128;

In [9]:
namedWindow("corners", WINDOW_GUI_EXPANDED); // Create a window
createTrackbar("Threshold: ", "corners", &slider_value, 255);

Run the event loop

In [10]:
while (waitKey(1) != 27)
{
    // Get a new frame
    Mat test_image;
    webcam >> test_image;
    
    // Match keypoint
    Mat image_with_corners;
    Mat detected_corners;
    detectCorners(slider_value, test_image, image_with_corners, detected_corners);
    
    // Combine the two images in one
    Mat concat;
    cvtColor(detected_corners, detected_corners, COLOR_GRAY2BGR);

    hconcat(image_with_corners, detected_corners, concat);
    
    imshow("corners", concat);
}

destroyAllWindows();

![Corners](../harris.gif)

Stop the camera

In [12]:
webcam.release();